# Tensor Flow 맛보기!

## MNIST

### Your name :     

### Kaggle Link for exercise :
* [Digit Recognizer](https://www.kaggle.com/c/digit-recognizer) : 

#### 실습목표<br>
1. TensorFlow의 모델링 아이디어를 이해한다.
2. (Optional)모든 코드를 이해하고 사용할 줄 안다.
---------------
Rayleigh Kim @ D:plus


In [0]:
'''
matplolib inline 명령어를 통해서
matplot으로 그리는 플롯들을 주피터 노트북 내에서 볼 수 있게 해준다.
포맷을 retina로 바꾸면 그래프의 화질이 훨씬 좋아진다.
'''
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

'''
라이브러리들을 불러오자.
'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import random as rd

from sklearn.metrics import accuracy_score

## Data Loading & Preparation

In [0]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()

In [0]:
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

In [0]:
'''
Ctrl+Enter를 이용하여
반복 실행 해보자!
'''

id = rd.randrange(0,10000)

print('id = {}'.format(id))
print('다음 그림은 숫자 {} 입니다.'.format(test_y[id]))
plt.imshow(test_x[id])
plt.show()

단순한 로지스틱 회귀, 뉴럴넷은 28x28의 2차원 데이터를 인풋으로 받지 못한다.

이미지[28 by 28] 를 납작한 array [784] 로 바꿔줄 필요가 있다.

In [0]:
train_x = train_x.reshape([train_x.shape[0],-1])
test_x = test_x.reshape([test_x.shape[0],-1])

print(train_x.shape, test_x.shape)

이미지의 값이기 때문에 0 ~ 255 사이의 값을 갖는다.

전부 0~1사이의 값을 갖도록 scale을 조정하자!

In [0]:
print('max :', train_x.max(),'  min :', train_x.min())

In [0]:
max_num = train_x.max()

train_x = train_x/max_num
test_x = test_x/max_num

print('max :', train_x.max(),'  min :', train_x.min())

## Logistic(Softmax) Regression Scikit-Learn Ver.

In [0]:
from sklearn.neural_network import MLPClassifier

In [0]:
logistic = MLPClassifier(hidden_layer_sizes=(),
                         early_stopping=True ,verbose=True)

In [0]:
logistic.fit(train_x, train_y)

In [0]:
logistic_pred_train = logistic.predict(train_x)
logistic_pred_test = logistic.predict(test_x)

logi_train_accuracy = accuracy_score(train_y, logistic_pred_train)
logi_test_accuracy = accuracy_score(test_y, logistic_pred_test)

print('로지스틱 리그레션')
print('트레이닝 정확도 : {:.2f}%'.format(logi_train_accuracy*100))
print('테스트 정확도 : {:.2f}%'.format(logi_test_accuracy*100))

In [0]:
'''
성능 확인을 위해
Ctrl+Enter를 이용하여
반복 실행 해보자!
'''

id = rd.randrange(0,10000)

print('id = {}'.format(id))
print('다음 그림은 숫자 {} 입니다.'.format(test_y[id]))
print('모델의 예측 : {}'.format(logistic_pred_test[id]))
if test_y[id] == logistic_pred_test[id] :
    print('정답입니다')
else : 
    print('틀렸어요')
plt.imshow(test_x[id].reshape([28,-1]))
plt.show()

In [0]:
'''
틀린 것만 관찰해보자!

Ctrl+Enter를 이용하여
반복 실행 해보자!
'''

true_false = (test_y == logistic_pred_test)
f_id = np.where(true_false == False)[0]
f_n = len(f_id)

id = f_id[rd.randrange(0,f_n)]

 
print('id = {}'.format(id))
print('다음 그림은 숫자 {} 입니다.'.format(test_y[id]))
print('모델의 예측 : {}'.format(logistic_pred_test[id]))
if test_y[id] == logistic_pred_test[id] :
    print('정답입니다')
else : 
    print('틀렸어요')
plt.imshow(test_x[id].reshape([28,-1]))
plt.show()

## Logisitic Regression Tensorflow Ver.

In [0]:
print(train_x.shape, train_y.shape)

In [0]:
# One-hot Encoding

train_y = np.eye(10)[train_y]
test_y = np.eye(10)[test_y]

In [0]:
print(train_y.shape, test_y.shape)

In [0]:
num_examples = train_y.shape[0]

In [0]:
'''
직접 짤 수 있어야 합니다!

인풋의 Shape = [None, 784]
아웃풋의 Shape = [None, 10]
Weight 의 Shape = [784, 10]
Bias 의 Shape = [10]
'''
# Graph Clear & Make your Graph reproducible
tf.reset_default_graph()
tf.set_random_seed(2017)

# Hyper parameters & Option
learning_rate = 0.01

# Place Holders
X = tf.placeholder(dtype=tf.float32, shape=[None, 784])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 10])

# Weights & bias
W = tf.Variable(tf.random_normal([784, 10]))
b = tf.Variable(tf.random_normal([10]))

# Hypothesis
logits = tf.add(tf.matmul(X, W), b)
Y_prob = tf.nn.softmax(logits)

# Accuracy
n_of_correct = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy =  tf.reduce_mean(tf.cast(n_of_correct, tf.float32))

# Cost(loss) function & Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(200):
    # 학습 진행
    sess.run(optimizer, feed_dict={X: train_x, Y: train_y})

    if ( (i+1) % 5 == 0): 
        # 학습 상황 디스플레이
        tr_a = sess.run(accuracy, feed_dict={X: train_x, Y: train_y})
        print("{}th, training accuracy: {:.3f}%".format(i+1, tr_a * 100))

In [0]:
logistic1_pred_train = sess.run(Y_prob, feed_dict={X:train_x})
logistic1_pred_test = sess.run(Y_prob, feed_dict={X:test_x})

single_pred_train = logistic1_pred_train.argmax(axis=1)
single_pred_test = logistic1_pred_test.argmax(axis=1)


logi_train_accuracy = accuracy_score(train_y.argmax(axis=1), single_pred_train)
logi_test_accuracy = accuracy_score(test_y.argmax(axis=1), single_pred_test)


print('로지스틱 리그레션')
print('트레이닝 정확도 : {:.2f}%'.format(logi_train_accuracy*100))
print('테스트 정확도 : {:.2f}%'.format(logi_test_accuracy*100))

In [0]:
'''
성능 확인을 위해
Ctrl+Enter를 이용하여
반복 실행 해보자!
'''

id = rd.randrange(0,10000)

print('id = {}'.format(id))
print('다음 그림은 숫자 {} 입니다.'.format(test_y.argmax(axis=1)[id]))
print('모델의 예측 : {}'.format(single_pred_test[id]))
print('모델의 카테고리별 확률 : {}'.format(np.floor(logistic1_pred_test[id]*100)))
if test_y.argmax(axis=1)[id] == single_pred_test[id] :
    print('정답입니다')
else : 
    print('틀렸어요')
plt.imshow(test_x[id].reshape([28,-1]))
plt.show()

In [0]:
'''
틀린 것만 관찰해보자!

Ctrl+Enter를 이용하여
반복 실행 해보자!
'''

true_false = (test_y.argmax(axis=1) == single_pred_test)
f_id = np.where(true_false == False)[0]
f_n = len(f_id)

id = f_id[rd.randrange(0,f_n)]


print('id = {}'.format(id))
print('다음 그림은 숫자 {} 입니다.'.format(test_y.argmax(axis=1)[id]))
print('모델의 예측 : {}'.format(single_pred_test[id]))
print('모델의 카테고리별 확률 : {}'.format(np.floor(logistic1_pred_test[id]*100)))
if test_y.argmax(axis=1)[id] == single_pred_test[id] :
    print('정답입니다')
else : 
    print('틀렸어요')
plt.imshow(test_x[id].reshape([28,-1]))
plt.show()

## (Optional) Artificial Neural Networks with Scikit-Learn

In [0]:
ANN = MLPClassifier(hidden_layer_sizes=(512,),
                         early_stopping=True ,verbose=True)

In [0]:
ANN.fit(train_x, train_y)

In [0]:
ann_pred_train = ANN.predict(train_x)
ann_pred_test = ANN.predict(test_x)

ann_train_accuracy = accuracy_score(train_y, ann_pred_train)
ann_test_accuracy = accuracy_score(test_y, ann_pred_test)

print('인공신경망')
print('트레이닝 정확도 : {:.2f}%'.format(ann_train_accuracy*100))
print('테스트 정확도 : {:.2f}%'.format(ann_test_accuracy*100))

In [0]:
'''
성능 확인을 위해
Ctrl+Enter를 이용하여
반복 실행 해보자!
'''

id = rd.randrange(0,10000)

print('id = {}'.format(id))
print('다음 그림은 숫자 {} 입니다.'.format(test_y[id]))
print('모델의 예측 : {}'.format(ann_pred_test[id]))
if test_y[id] == ann_pred_test[id] :
    print('정답입니다')
else : 
    print('틀렸어요')
plt.imshow(test_x[id].reshape([28,-1]))
plt.show()

In [0]:
'''
틀린 것만 관찰해보자!

Ctrl+Enter를 이용하여
반복 실행 해보자!
'''

true_false = (test_y == ann_pred_test)
f_id = np.where(true_false == False)[0]
f_n = len(f_id)

id = f_id[rd.randrange(0,f_n)]

 
print('id = {}'.format(id))
print('다음 그림은 숫자 {} 입니다.'.format(test_y[id]))
print('모델의 예측 : {}'.format(ann_pred_test[id]))
if test_y[id] == ann_pred_test[id] :
    print('정답입니다')
else : 
    print('틀렸어요')
plt.imshow(test_x[id].reshape([28,-1]))
plt.show()

# (Very Optional) 새로운 데이터로, 처음부터 실습!

들어는 보았는가 Fashion MNIST?

In [0]:
'''
matplolib inline 명령어를 통해서
matplot으로 그리는 플롯들을 주피터 노트북 내에서 볼 수 있게 해준다.
포맷을 retina로 바꾸면 그래프의 화질이 훨씬 좋아진다.
'''
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

'''
라이브러리들을 불러오자.
'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import random as rd

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [0]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.fashion_mnist.load_data()

In [0]:
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

In [0]:
'''
Ctrl+Enter를 이용하여
반복 실행 해보자!
'''

labels = ['T-shirt/Top', 'Trouser', 'Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

id = rd.randrange(0,10000)
lid = test_y[id]

print('id = {}'.format(id))
print('다음 그림은 {} 입니다.'.format(labels[lid]) )
plt.imshow(test_x[id])
plt.show()

In [0]:
###### Now, Your Turn!